# Reproduce results of the paper

This notebook allows you to reproduce the main experiments of the paper. Run all cells in order, following any instructions that are given. Only make changes to cells if instructions permit.

First, please ensure the dataset is downloaded and placed it the project folder. The dataset is obtained from: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

### Install Requirements

Next, ensure all requirements are installed

In [ ]:
! pip install -r requirements.txt

### Training Preparation

Import the relevant modules

In [ ]:
import torch
import numpy as np
import random

import utils
from utils.data_loaders import create_loaders, quick_loader_test, full_loader_test
from utils.training import create_model_and_optimizer, train_model
from utils.evaluation import get_training_results, evaluate_model
from utils.visualisations import visualise_model_predictions

Set random seed for reproducability

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

The following cell allows you to choose which model to train. You may also change hyperparameters. Provided hyperparameters are those used in the main experiment of the paper.

In [ ]:
# MAIN CHOICE
# -----------

# Choose model architecture
# Choose one of: 'SmallCNN' 'CamCNN' 'LargeCNN' 'ResNet3fc' 'ResNet1fc' (must be identical string)
model_type = 'CamCNN' 

# Directory chest-xray data is stored in
data_dir = 'chest_xray'

# Directory to save models and logs
exp_dir = 'experiments/reproduce'

# ADDITIONAL OPTIONS - recommended to leave these at default values to properly recreate experiments
# ------------------

# Choose to use original data split
use_og_split = True # If False, training data is split into train/val/test

# Perform augmentation on training data
augment_data = True

# Weight sampling of training data so both classes are sampled equally
weight_samples = True

# Use dropout in fully-connected layers of model
use_dropout = True

# Use adam optimizer. If False, SGD is used
use_adam = True

learning_rate = 0.001 # Select learning rate
batch_size = 16       # Select batch size
epochs = 50           # Select number of training epochs

Obtain the dataloaders

In [ ]:
train_loader, valid_loader, test_loader = create_loaders(data_dir, augment_data, weight_samples, batch_size, use_og_split, num_workers=2)

Perform a quick visual test of the training dataloader

In [ ]:
quick_loader_test(train_loader)

Create the model

In [ ]:
model, optimizer = create_model_and_optimizer(model_type, use_dropout, use_adam, learning_rate)

### Training

Finally, begin training the model !!

Expect each epoch to take somewhere between 5-10 mins with a GPU

In [ ]:
train_model(model, optimizer, train_loader, valid_loader, epochs, exp_dir)

### Evaluation

Now we will evaluate the model with the lowest validation loss.

First, lets display the training results and load in the best model.

In [ ]:
best_model = get_training_results(model_type, exp_dir, use_dropout)

Now the best model will be evaluated on the test data

In [ ]:
evaluate_model(best_model, test_loader, exp_dir)

### Model Interpretation

Now we will visualise which areas of the image the model is focusing on when making predictions

If the model is a 'SmallCNN' or a ResNet, the Grad-CAM technique is used.

If the model is a 'CAM_CNN' then the CAM method is used.

In [ ]:
visualise_model_predictions(best_model, test_loader, exp_dir)